In [1]:
import mdptoolbox
import numpy as np
import scipy as sc

In [2]:
A = 2
S = 3

In [5]:
def states_actions():
    states_dict = { 
        # Terminal States
        'accept' : 0,
        'reject' : 1,
        # continuation
        'continue': 2}
    
    action_dict = {'higher': 0, 'same': 1}
    
    return states_dict, action_dict

In [69]:
# transition matrix
def transition(offered_price, sticker_price):
    
    # bid higher
    new_offered_price = offered_price + 20000
    
    a1s11 = np.min([0.333 + (new_offered_price - sticker_price)/sticker_price, 1])
    a1s11 = np.max([0, a1s11])
    
    
    a1s21 = np.min([0.333 - (new_offered_price - sticker_price)/sticker_price, 1])
    a1s21 = np.max([a1s21, 0])
    
    # bid same      
    a2s11 = np.min([0.333 + (offered_price - sticker_price)/sticker_price, 1])
    a2s11 = np.max([0, a2s11])
    
    
    a2s21 = np.min([0.333 - (offered_price - sticker_price)/sticker_price, 1])
    a2s21 = np.max([a2s21, 0])
    
    
    a1 = np.array([[1,0,0], [0,1,0], [a1s11, a1s21, 1 - a1s21 - a1s11]])
    a2 = np.array([[1,0,0], [0,1,0], [a2s11, a2s21, 1- a2s21 -a2s11 ]])
    
    transition_list = [a1, a2]
    
    return transition_list

In [41]:
transition(460000,400000)

[array([[1.   , 0.   , 0.   ],
        [0.   , 1.   , 0.   ],
        [0.533, 0.133, 0.334]]),
 array([[1.   , 0.   , 0.   ],
        [0.   , 1.   , 0.   ],
        [0.483, 0.183, 0.334]])]

In [6]:
def rewards(budget, offered_price):
        
    # terminal rewards
    
    price_to_budget = (budget - offered_price)/budget 
    
    # if budget > price, p2b > 0
    # if budget < price, p2b < 0
    
    # for state 0 (accept)
    reward_zero = 1 + (price_to_budget*100)/25
    R = np.ones(A)*reward_zero

    # for state 1 (reject)
    reward_one = 0
    row_one = np.ones(A)*reward_one
    R = np.vstack([R, row_one])

    # state 2 (continue)
    s2_a0 = price_to_budget
    s2_a1 = 0
    row_two = np.array([s2_a0,s2_a1])
    R = np.vstack([R, row_two])
    
    return R        

In [61]:
budget = 450000
offered_price = 450000
sticker_price = 100000

P = transition(offered_price, sticker_price)
R = rewards(budget, offered_price)
pi = mdptoolbox.mdp.PolicyIteration(P, R, 0.9, max_iter = 500)
pi.run()

In [85]:
budget = 80000
offered_price = 90000
sticker_price = 100000

P = transition(offered_price, sticker_price)
R = rewards(budget, offered_price)
pi = mdptoolbox.mdp.QLearning(P, R, 0.9)
pi.run()

In [86]:
print(pi.policy)
pi.V

(0, 0, 1)


(4.996746357004661, 0.0, 0.8163400356456989)